In [1]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_model_optimization as tfmot
from distiller import Distiller
#import datetime

import tensorflow_model_optimization as tfmot
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

%load_ext tensorboard

In [2]:
tf.random.set_seed(1234)

In [3]:
# Load CIFAR10 dataset
cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(256),
        layers.Dropout(0.5),
        layers.Dense(256),
        layers.Dense(10),
    ],)

quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(keras.models.clone_model(model))
q_aware_16_model = quantize_model(keras.models.clone_model(model))

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(
    train_images,
    train_labels,
    epochs=5,
    validation_split=0.1,
)
#%tensorboard --logdir logs

Epoch 1/5
1407/1407 [==============================] - 52s 36ms/step - loss: 1.4882 - accuracy: 0.4616 - val_loss: 1.1876 - val_accuracy: 0.5832
Epoch 2/5
1407/1407 [==============================] - 54s 38ms/step - loss: 1.1020 - accuracy: 0.6167 - val_loss: 0.9802 - val_accuracy: 0.6550
Epoch 3/5
1407/1407 [==============================] - 51s 36ms/step - loss: 0.9324 - accuracy: 0.6802 - val_loss: 0.9120 - val_accuracy: 0.6868
Epoch 4/5
1407/1407 [==============================] - 45s 32ms/step - loss: 0.8112 - accuracy: 0.7191 - val_loss: 0.9338 - val_accuracy: 0.6882
Epoch 5/5
1407/1407 [==============================] - 43s 30ms/step - loss: 0.7122 - accuracy: 0.7552 - val_loss: 0.8434 - val_accuracy: 0.7230


In [4]:
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.fit(
    train_images,
    train_labels,
    epochs=5,
    validation_split=0.1,
)

Epoch 1/5
1407/1407 [==============================] - 69s 48ms/step - loss: 1.5950 - accuracy: 0.4151 - val_loss: 1.2766 - val_accuracy: 0.5344
Epoch 2/5
1407/1407 [==============================] - 70s 50ms/step - loss: 1.1955 - accuracy: 0.5808 - val_loss: 1.0152 - val_accuracy: 0.6414
Epoch 3/5
1407/1407 [==============================] - 70s 50ms/step - loss: 1.0034 - accuracy: 0.6532 - val_loss: 0.9542 - val_accuracy: 0.6740
Epoch 4/5
1407/1407 [==============================] - 67s 47ms/step - loss: 0.8779 - accuracy: 0.6958 - val_loss: 0.9116 - val_accuracy: 0.6876
Epoch 5/5
1407/1407 [==============================] - 69s 49ms/step - loss: 0.7724 - accuracy: 0.7348 - val_loss: 0.8553 - val_accuracy: 0.7098


In [5]:
q_aware_16_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_16_model.fit(
    train_images,
    train_labels,
    epochs=5,
    validation_split=0.1,
)

Epoch 1/5
1407/1407 [==============================] - 69s 49ms/step - loss: 1.6010 - accuracy: 0.4177 - val_loss: 1.3739 - val_accuracy: 0.5240
Epoch 2/5
1407/1407 [==============================] - 68s 49ms/step - loss: 1.1834 - accuracy: 0.5833 - val_loss: 1.0387 - val_accuracy: 0.6366
Epoch 3/5
1407/1407 [==============================] - 68s 49ms/step - loss: 0.9920 - accuracy: 0.6552 - val_loss: 0.9369 - val_accuracy: 0.6812
Epoch 4/5
1407/1407 [==============================] - 71s 51ms/step - loss: 0.8603 - accuracy: 0.7015 - val_loss: 0.9020 - val_accuracy: 0.6964
Epoch 5/5
1407/1407 [==============================] - 67s 48ms/step - loss: 0.7552 - accuracy: 0.7366 - val_loss: 0.9838 - val_accuracy: 0.6758


In [6]:
# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(10),
    ],
    name="student",
)
# Clone student for later comparison
student_scratch = keras.models.clone_model(student)
q_aware_student = quantize_model(keras.models.clone_model(student))
q_aware_16_student = quantize_model(keras.models.clone_model(student))

# Train student
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(train_images, train_labels, epochs=3)
student_scratch.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 20s 13ms/step - loss: 1.4744 - sparse_categorical_accuracy: 0.4623
Epoch 2/3
1563/1563 [==============================] - 20s 13ms/step - loss: 1.0863 - sparse_categorical_accuracy: 0.6217
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - loss: 0.8975 - sparse_categorical_accuracy: 0.6912A: 1s - loss: 0.9035 - sparse_categorica


[0.8975274562835693, 0.6912000179290771]

In [7]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(train_images, train_labels, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 29s 18ms/step - sparse_categorical_accuracy: 0.4758 - student_loss: 1.4452 - distillation_loss: 0.0020 0s - sparse_categorical_accuracy: 0.4724 - student_loss: 1.4540 - distillatio
Epoch 2/3
1563/1563 [==============================] - 29s 19ms/step - sparse_categorical_accuracy: 0.6268 - student_loss: 1.0648 - distillation_loss: 0.0014
Epoch 3/3
313/313 [==============================] - 2s 5ms/step - sparse_categorical_accuracy: 0.6801 - student_loss: 0.9385


[0.6801000237464905, 1.1796770095825195]

In [8]:
#quantize -> distill
# Initialize and compile distiller
qd = Distiller(student=student, teacher=q_aware_model)
qd.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
qd.fit(train_images, train_labels, epochs=3)

# Evaluate student on test dataset
qd.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 45s 28ms/step - sparse_categorical_accuracy: 0.7292 - student_loss: 0.7833 - distillation_loss: 0.0011
Epoch 2/3
1563/1563 [==============================] - 39s 25ms/step - sparse_categorical_accuracy: 0.7603 - student_loss: 0.6891 - distillation_loss: 0.0013
Epoch 3/3
313/313 [==============================] - 1s 5ms/step - sparse_categorical_accuracy: 0.7062 - student_loss: 0.8889


[0.7062000036239624, 0.7706981301307678]

In [9]:
# Initialize and compile distiller
dq = Distiller(student=q_aware_student, teacher=model)
dq.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
dq.fit(train_images, train_labels, epochs=3)

# Evaluate student on test dataset
dq.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 38s 24ms/step - sparse_categorical_accuracy: 0.4443 - student_loss: 1.5273 - distillation_loss: 0.0022
Epoch 2/3
1563/1563 [==============================] - 37s 24ms/step - sparse_categorical_accuracy: 0.6036 - student_loss: 1.1250 - distillation_loss: 0.0015 30s - spar - ETA: 25s - sparse_c - ETA: 22s - sparse_categorical_accuracy: 0.5832 - student_loss: 1.1799 - distillation_ - ETA: 22 - ETA: 9s - sparse_categorical_accuracy: 0.5938 - student_loss:  - ETA: 7s - sparse_categorical_accuracy: 0.5966 - student_loss: 1.1415 - dist
Epoch 3/3
313/313 [==============================] - 2s 6ms/step - sparse_categorical_accuracy: 0.6781 - student_loss: 0.9330


[0.6780999898910522, 1.13441801071167]

In [10]:
# Initialize and compile distiller
dq_16 = Distiller(student=q_aware_16_student, teacher=model)
dq_16.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
dq_16.fit(train_images, train_labels, epochs=3)

# Evaluate student on test dataset
dq_16.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 38s 23ms/step - sparse_categorical_accuracy: 0.4470 - student_loss: 1.5181 - distillation_loss: 0.0022 5s - sparse_categorical_accuracy: 0.4276 - student_loss: 1.5644 - dist - ETA: 4s - sparse_categorical_accuracy: 0.4315 - st - ETA: 1s - sparse_categorical_accuracy: 0.4416 - student_loss: 1.5305
Epoch 2/3
1563/1563 [==============================] - 38s 24ms/step - sparse_categorical_accuracy: 0.6049 - student_loss: 1.1249 - distillation_loss: 0.0015 5s - sparse_categorical_accuracy: 0.5994 - student_loss: 1.1391 - distillation_loss: 0.00 - ETA: 5s - s
Epoch 3/3
313/313 [==============================] - 2s 6ms/step - sparse_categorical_accuracy: 0.6678 - student_loss: 0.9671


[0.6678000092506409, 1.07920241355896]

In [11]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.704200029373169
Saved baseline model to: C:\Users\yeswe\AppData\Local\Temp\tmpn17kjfgd.h5


In [12]:
_, q_aware_model_accuracy = q_aware_model.evaluate(
    test_images, test_labels, verbose=0)

print('Q Aware test accuracy:', q_aware_model_accuracy)

_, q_aware_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(q_aware_model, q_aware_keras_file, include_optimizer=False)
print('Saved Q Aware model to:', q_aware_keras_file)

Q Aware test accuracy: 0.6945000290870667
Saved Q Aware model to: C:\Users\yeswe\AppData\Local\Temp\tmp6af231q7.h5


In [13]:
distiller_accuracy, _ = distiller.evaluate(
    test_images, test_labels, verbose=0)

print('Distiller test accuracy:', distiller_accuracy)

_, distiller_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(distiller.student, distiller_keras_file, include_optimizer=False)
print('Saved Distiller model to:', distiller_keras_file)

Distiller test accuracy: 0.7062000036239624
Saved Distiller model to: C:\Users\yeswe\AppData\Local\Temp\tmpxjw3_vhe.h5


In [14]:
qd_accuracy, _ = qd.evaluate(
    test_images, test_labels, verbose=0)

print('Quant -> Distill test accuracy:', qd_accuracy)

_, qd_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(qd.student, qd_keras_file, include_optimizer=False)
print('Saved Quant -> Distill model to:', qd_keras_file)

Quant -> Distill test accuracy: 0.7062000036239624
Saved Quant -> Distill model to: C:\Users\yeswe\AppData\Local\Temp\tmpfrzv327s.h5


In [15]:
dq_accuracy, _ = dq.evaluate(
    test_images, test_labels, verbose=0)

print('Distill -> Quant test accuracy:', dq_accuracy)

_, dq_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(dq.student, dq_keras_file, include_optimizer=False)
print('Saved Distill -> Quant model to:', dq_keras_file)

Distill -> Quant test accuracy: 0.6780999898910522
Saved Distill -> Quant model to: C:\Users\yeswe\AppData\Local\Temp\tmpu_xd6kz4.h5


In [16]:
def get_gzipped_model_size(file):
    # Returns size of gzipped model, in bytes.
    import os
    import zipfile

    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

In [17]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped Q Aware Keras model: %.2f bytes" % (get_gzipped_model_size(q_aware_keras_file)))
print("Size of gzipped Distiller Keras model: %.2f bytes" % (get_gzipped_model_size(distiller_keras_file)))
print("Size of gzipped Quant -> Distill Keras model: %.2f bytes" % (get_gzipped_model_size(qd_keras_file)))
print("Size of gzipped Distill -> Quant Keras model: %.2f bytes" % (get_gzipped_model_size(dq_keras_file)))

Size of gzipped baseline Keras model: 5074958.00 bytes
Size of gzipped Q Aware Keras model: 5083737.00 bytes
Size of gzipped Distiller Keras model: 1280610.00 bytes
Size of gzipped Quant -> Distill Keras model: 1280608.00 bytes
Size of gzipped Distill -> Quant Keras model: 1284811.00 bytes


In [30]:
model_converter = tf.lite.TFLiteConverter.from_keras_model(model)
model_tflite_model = model_converter.convert()

q_aware_converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
q_aware_converter.optimizations = [tf.lite.Optimize.DEFAULT]
q_aware_tflite_model = q_aware_converter.convert()

q8_converter = tf.lite.TFLiteConverter.from_keras_model(model)
q8_converter.optimizations = [tf.lite.Optimize.DEFAULT]
q8_tflite_model = q8_converter.convert()

q16_converter = tf.lite.TFLiteConverter.from_keras_model(model)#q_aware_16_model)
q16_converter.optimizations = [tf.lite.Optimize.DEFAULT]
q16_converter.target_spec.supported_types = [tf.float16]
q16_tflite_model = q16_converter.convert()

distiller_converter = tf.lite.TFLiteConverter.from_keras_model(distiller.student)
#distiller_converter.optimizations = [tf.lite.Optimize.DEFAULT]
distiller_tflite_model = distiller_converter.convert()

qd_converter = tf.lite.TFLiteConverter.from_keras_model(qd.student)
#qd_converter.optimizations = [tf.lite.Optimize.DEFAULT]
qd_tflite_model = qd_converter.convert()

dq_converter = tf.lite.TFLiteConverter.from_keras_model(dq.student)
dq_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dq_tflite_model = dq_converter.convert()

dq_8_converter = tf.lite.TFLiteConverter.from_keras_model(distiller.student)#dq_16.student)
dq_8_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dq_8_tflite_model = dq_8_converter.convert()

dq_16_converter = tf.lite.TFLiteConverter.from_keras_model(distiller.student)#dq_16.student)
dq_16_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dq_16_converter.target_spec.supported_types = [tf.float16]
dq_16_tflite_model = dq_16_converter.convert()


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmp53g6c6kd\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmp53g6c6kd\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmp11qc70ws\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmp11qc70ws\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmpk5nb61i9\assets


...

INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmp5tpd4aw9\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmp0qd3_beq\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmp0qd3_beq\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmpnhvzff_t\assets


INFO:tensorflow:Assets written to: C:\Users\yeswe\AppData\Local\Temp\tmpnhvzff_t\assets


In [31]:
# Measure sizes of models.
_, model_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')
_, q16_file = tempfile.mkstemp('.tflite')
_, q8_file = tempfile.mkstemp('.tflite')
_, distiller_file = tempfile.mkstemp('.tflite')
_, qd_file = tempfile.mkstemp('.tflite')
_, dq_file = tempfile.mkstemp('.tflite')
_, dq_8_file = tempfile.mkstemp('.tflite')
_, dq_16_file = tempfile.mkstemp('.tflite')

In [32]:
with open(quant_file, 'wb') as f:
  f.write(q_aware_tflite_model)

with open(q8_file, 'wb') as f:
  f.write(q8_tflite_model)

with open(q16_file, 'wb') as f:
  f.write(q16_tflite_model)

with open(model_file, 'wb') as f:
  f.write(model_tflite_model)

with open(distiller_file, 'wb') as f:
  f.write(distiller_tflite_model)

with open(qd_file, 'wb') as f:
  f.write(qd_tflite_model)

with open(dq_file, 'wb') as f:
  f.write(dq_tflite_model)

with open(dq_8_file, 'wb') as f:
  f.write(dq_8_tflite_model)

with open(dq_16_file, 'wb') as f:
  f.write(dq_16_tflite_model)

In [33]:
print("Original model in Mb:", os.path.getsize(model_file) / float(2**20))
print("Q Aware model in Mb:", os.path.getsize(quant_file) / float(2**20))
print("Quantized 8 model in Mb:", os.path.getsize(q8_file) / float(2**20))
print("Quantized 16 model in Mb:", os.path.getsize(q16_file) / float(2**20))
print("Distiller model in Mb:", os.path.getsize(distiller_file) / float(2**20))
print("Quant -> Distill model in Mb:", os.path.getsize(qd_file) / float(2**20))
print("Distill -> Quant model in Mb:", os.path.getsize(dq_file) / float(2**20))
print("Distill -> Quant 8 model in Mb:", os.path.getsize(dq_8_file) / float(2**20))
print("Distill -> Quant 16 model in Mb:", os.path.getsize(dq_16_file) / float(2**20))

Original model in Mb: 5.218955993652344
Q Aware model in Mb: 1.3317031860351562
Quantized 8 model in Mb: 1.32635498046875
Quantized 16 model in Mb: 2.613739013671875
Distiller model in Mb: 1.3147850036621094
Quant -> Distill model in Mb: 1.3147850036621094
Distill -> Quant model in Mb: 0.34481048583984375
Distill -> Quant 8 model in Mb: 0.3414459228515625
Distill -> Quant 16 model in Mb: 0.6615447998046875


In [34]:
print('original accuracy: ' + str(baseline_model_accuracy))
print('q aware accuracy: ' + str(q_aware_model_accuracy))
print('distiller accuracy: ' + str(distiller_accuracy))
print('qd accuracy: ' + str(qd_accuracy))
print('dq accuracy: ' + str(dq_accuracy))

original accuracy: 0.704200029373169
q aware accuracy: 0.6945000290870667
distiller accuracy: 0.7062000036239624
qd accuracy: 0.7062000036239624
dq accuracy: 0.6780999898910522


In [35]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for i, test_image in enumerate(test_images):
        if i%100 == 0:
            print('got to ' + str(i))
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
        
    # Compare prediction results with ground truth labels to calculate accuracy.
    accurate_count = 0
    for index in range(len(prediction_digits)):
        if prediction_digits[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_digits)

    return accuracy

In [36]:
original_interpreter = tf.lite.Interpreter(model_content=model_tflite_model)
original_interpreter.allocate_tensors()

q_aware_interpreter = tf.lite.Interpreter(model_content=q_aware_tflite_model)
q_aware_interpreter.allocate_tensors()

q8_interpreter = tf.lite.Interpreter(model_content=q8_tflite_model)
q8_interpreter.allocate_tensors()

q16_interpreter = tf.lite.Interpreter(model_content=q16_tflite_model)
q16_interpreter.allocate_tensors()

distiller_interpreter = tf.lite.Interpreter(model_content=distiller_tflite_model)
distiller_interpreter.allocate_tensors()

qd_interpreter = tf.lite.Interpreter(model_content=qd_tflite_model)
qd_interpreter.allocate_tensors()

dq_interpreter = tf.lite.Interpreter(model_content=dq_tflite_model)
dq_interpreter.allocate_tensors()

dq_8_interpreter = tf.lite.Interpreter(model_content=dq_8_tflite_model)
dq_8_interpreter.allocate_tensors()

dq_16_interpreter = tf.lite.Interpreter(model_content=dq_16_tflite_model)
dq_16_interpreter.allocate_tensors()

In [37]:
q_aware_accuracy = evaluate_model(q_aware_interpreter)

got to 0
got to 100
got to 200
got to 300
got to 400
got to 500
got to 600
got to 700
got to 800
got to 900
got to 1000
got to 1100
got to 1200
got to 1300
got to 1400
got to 1500
got to 1600
got to 1700
got to 1800
got to 1900
got to 2000
got to 2100
got to 2200
got to 2300
got to 2400
got to 2500
got to 2600
got to 2700
got to 2800
got to 2900
got to 3000
got to 3100
got to 3200
got to 3300
got to 3400
got to 3500
got to 3600
got to 3700
got to 3800
got to 3900
got to 4000
got to 4100
got to 4200
got to 4300
got to 4400
got to 4500
got to 4600
got to 4700
got to 4800
got to 4900
got to 5000
got to 5100
got to 5200
got to 5300
got to 5400
got to 5500
got to 5600
got to 5700
got to 5800
got to 5900
got to 6000
got to 6100
got to 6200
got to 6300
got to 6400
got to 6500
got to 6600
got to 6700
got to 6800
got to 6900
got to 7000
got to 7100
got to 7200
got to 7300
got to 7400
got to 7500
got to 7600
got to 7700
got to 7800
got to 7900
got to 8000
got to 8100
got to 8200
got to 8300
got 

In [38]:
print('evaluating q8...')
q8_accuracy = evaluate_model(q8_interpreter)

evaluating q8...
got to 0
got to 100
got to 200
got to 300
got to 400
got to 500
got to 600
got to 700
got to 800
got to 900
got to 1000
got to 1100
got to 1200
got to 1300
got to 1400
got to 1500
got to 1600
got to 1700
got to 1800
got to 1900
got to 2000
got to 2100
got to 2200
got to 2300
got to 2400
got to 2500
got to 2600
got to 2700
got to 2800
got to 2900
got to 3000
got to 3100
got to 3200
got to 3300
got to 3400
got to 3500
got to 3600
got to 3700
got to 3800
got to 3900
got to 4000
got to 4100
got to 4200
got to 4300
got to 4400
got to 4500
got to 4600
got to 4700
got to 4800
got to 4900
got to 5000
got to 5100
got to 5200
got to 5300
got to 5400
got to 5500
got to 5600
got to 5700
got to 5800
got to 5900
got to 6000
got to 6100
got to 6200
got to 6300
got to 6400
got to 6500
got to 6600
got to 6700
got to 6800
got to 6900
got to 7000
got to 7100
got to 7200
got to 7300
got to 7400
got to 7500
got to 7600
got to 7700
got to 7800
got to 7900
got to 8000
got to 8100
got to 8200

In [39]:
print('evaluating original...')
original_accuracy = evaluate_model(original_interpreter)
#print('evaluating q aware...')
#q_aware_accuracy = evaluate_model(q_aware_interpreter)
#print('evaluating q8...')
#q8_accuracy = evaluate_model(q8_interpreter)
print('evaluating q16...')
q16_accuracy = evaluate_model(q16_interpreter)
print('evaluating distilled...')
distiller_accuracy = evaluate_model(distiller_interpreter)
print('evaluating qd...')
qd_accuracy = evaluate_model(qd_interpreter)
print('evaluating dq...')
dq_accuracy = evaluate_model(dq_interpreter)
print('evaluating dq 8...')
dq_8_accuracy = evaluate_model(dq_8_interpreter)
print('evaluating dq 16...')
dq_16_accuracy = evaluate_model(dq_16_interpreter)

evaluating original...
got to 0
got to 100
got to 200
got to 300
got to 400
got to 500
got to 600
got to 700
got to 800
got to 900
got to 1000
got to 1100
got to 1200
got to 1300
got to 1400
got to 1500
got to 1600
got to 1700
got to 1800
got to 1900
got to 2000
got to 2100
got to 2200
got to 2300
got to 2400
got to 2500
got to 2600
got to 2700
got to 2800
got to 2900
got to 3000
got to 3100
got to 3200
got to 3300
got to 3400
got to 3500
got to 3600
got to 3700
got to 3800
got to 3900
got to 4000
got to 4100
got to 4200
got to 4300
got to 4400
got to 4500
got to 4600
got to 4700
got to 4800
got to 4900
got to 5000
got to 5100
got to 5200
got to 5300
got to 5400
got to 5500
got to 5600
got to 5700
got to 5800
got to 5900
got to 6000
got to 6100
got to 6200
got to 6300
got to 6400
got to 6500
got to 6600
got to 6700
got to 6800
got to 6900
got to 7000
got to 7100
got to 7200
got to 7300
got to 7400
got to 7500
got to 7600
got to 7700
got to 7800
got to 7900
got to 8000
got to 8100
got t

got to 8200
got to 8300
got to 8400
got to 8500
got to 8600
got to 8700
got to 8800
got to 8900
got to 9000
got to 9100
got to 9200
got to 9300
got to 9400
got to 9500
got to 9600
got to 9700
got to 9800
got to 9900


In [40]:
print('original accuracy ', original_accuracy)
print('q aware accuracy ', q_aware_accuracy)
print('q8 accuracy ', q8_accuracy)
print('q16 accuracy ', q16_accuracy)
print('distiller accuracy ', distiller_accuracy)
print('qd accuracy ', qd_accuracy)
print('dq accuracy ', dq_accuracy)
print('dq 8 accuracy ', dq_8_accuracy)
print('dq 16 accuracy ', dq_16_accuracy)


original accuracy  0.7042
q aware accuracy  0.6955
q8 accuracy  0.7036
q16 accuracy  0.7042
distiller accuracy  0.7062
qd accuracy  0.7062
dq accuracy  0.6793
dq 8 accuracy  0.7059
dq 16 accuracy  0.7062
